<a href="https://colab.research.google.com/github/Rhysus85/MA5851_MasterClass1/blob/main/Video_to_image_POC_Rhys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Check if using GPU 
#google suggestion as I have set up a subscription to get dedicated GPUs
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [2]:
#Check memory usage/availability
#google suggestion as I have set up a subscription to get dedicated GPUs
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [3]:
#mount drive to access data 
#I used my drive for the POC
import os 
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [4]:
#set directory
os.chdir('/content/drive/MyDrive/Ocean_Infinity')
#os.listdir()

#output is:
#['Model',
# 'Output',
# 'Dataset',
# 'Raw Video',
# 'Raw Video Names.csv',
# 'Raw Dataset']


Process here is to take video, convert to images based on video fps and store in specific folder to match the visibility label.

In [10]:
#load libraries
import cv2 
import numpy as np
from google.colab.patches import cv2_imshow #directed by google to reduce crashing
import pandas as pd
import csv
import math
from os.path import exists

#libraries not yet used
#from glob import glob
#from tqdm import tqdm
#import math   # for mathematical operations
#import matplotlib.pyplot as plt    # for plotting the images
#%matplotlib inline
#from keras.preprocessing import image   # for preprocessing the images
#import numpy as np    # for mathematical operations
#from keras.utils import np_utils
#from skimage.transform import resize   # for resizing images
#from sklearn.model_selection import train_test_split



In [6]:
raw_video_path = './Raw Video/'
raw_image_path = './Raw Dataset/'
image_metadata_row = pd.DataFrame()     
image_metadata = pd.DataFrame()

In [7]:
image_cnt = 0
#POC direct filepath
poc_video = 'video_POC.mp4'

#part of testing iterations, setting directory
os.chdir('/content/drive/MyDrive/Ocean_Infinity')

for path, subdirs, files in os.walk(raw_video_path):
    #print("stage0")
    for name in files:
      #print("stage1")
      #for POC only
      if name == poc_video:
        #print("status: Video found")
        cap = cv2.VideoCapture((os.path.join(path, name)))
        #metadata for video
        video_fps = cap.get(5)
        video_width = cap.get(3)
        video_height = cap.get(4)
        video_name = name
        video_path = path
        video_size = math.trunc(os.path.getsize((os.path.join(path, name))))
          #print('%.2f'%(file_byte/1000/1000),1) #file size in mb              
        #create image output path
        label= path.rsplit('/', 1)[1]
        image_dataset_path = raw_image_path+label
        #print("ready to go")
        
        while(cap.isOpened()):
          ret, frame = cap.read()
          if ret == False:
            break
          frame_ID = cap.get(1)
          try:
            img_name = label+'_'+str(image_cnt)+'.jpg'
            cv2.imwrite((image_dataset_path+'/'+img_name),frame)
            image_cnt +=1   

            #create metadata rows for excel extract
            image_metadata_raw = [video_path,
                                  video_size,
                                  video_name,
                                  video_fps,
                                  video_width,
                                  video_height,
                                  image_dataset_path,
                                  label,
                                  frame_ID,
                                  img_name
                                  ]
            image_metadata_row = pd.DataFrame(image_metadata_raw).T       
            image_metadata = image_metadata.append(image_metadata_row)


          except:
            print("you have landed at an exception #1")
            continue

        cap.release()
        cv2.destroyAllWindows()
              
      else:
        continue


In [8]:
image_metadata

,0,1,2,3,4,5,6,7,8,9
0,./Raw Video/POC,212056,video_POC.mp4,30.0,1920.0,1080.0,./Raw Dataset/POC,POC,1.0,POC_0.jpg
0,./Raw Video/POC,212056,video_POC.mp4,30.0,1920.0,1080.0,./Raw Dataset/POC,POC,2.0,POC_1.jpg
0,./Raw Video/POC,212056,video_POC.mp4,30.0,1920.0,1080.0,./Raw Dataset/POC,POC,3.0,POC_2.jpg
0,./Raw Video/POC,212056,video_POC.mp4,30.0,1920.0,1080.0,./Raw Dataset/POC,POC,4.0,POC_3.jpg


In [9]:
#do this at the end
image_metadata.columns=['video_path',
                                'video_size',
                                'video_name',
                                'video_fps',
                                'video_width',
                                'video_height',
                                'image_dataset_path',
                                'label',
                                'frame_ID',
                                'img_name'
                                ]

image_metadata

,video_path,video_size,video_name,video_fps,video_width,video_height,image_dataset_path,label,frame_ID,img_name
0,./Raw Video/POC,212056,video_POC.mp4,30.0,1920.0,1080.0,./Raw Dataset/POC,POC,1.0,POC_0.jpg
0,./Raw Video/POC,212056,video_POC.mp4,30.0,1920.0,1080.0,./Raw Dataset/POC,POC,2.0,POC_1.jpg
0,./Raw Video/POC,212056,video_POC.mp4,30.0,1920.0,1080.0,./Raw Dataset/POC,POC,3.0,POC_2.jpg
0,./Raw Video/POC,212056,video_POC.mp4,30.0,1920.0,1080.0,./Raw Dataset/POC,POC,4.0,POC_3.jpg


In [11]:
#create the image metadata file
os.chdir('/content/drive/MyDrive/Ocean_Infinity')
filename = 'image_metadata.csv'

if os.path.exists(filename):
  image_metadata.to_csv(filename, mode='a', index=False, header=False)
  print("added new rows of data")
else:
  image_metadata.to_csv(filename, mode='a', index=False, header=True)
  print("created new file")


added new rows of data
